In [1]:
import pandas as pd
from settings import *
from load import *

In [30]:
file_name = path['data']['processed_data'] / 'additional_url_data.pickle'
data = load(file_name, dict())

data_col = load_col('url_host')

cats = list(data_col.cat.categories)
df = pd.DataFrame([[x] + list(reversed(x.split('.'))) for x in cats])

cols = df.columns
switcher = (((df[2] == 'turbopages') & (~df[3].isna())).astype('int32')) == 1
# switcher = (((df[0].str.contains('turbopages.org')) & (~df[3].isna())).astype('int32')) == 1
new_df = df[switcher][3].str.replace('--', '$special_char$', regex=False).str.replace('-', '.', regex=False).str.replace('$special_char$', '-', regex=False).str.split('.')
new_df = pd.DataFrame([reversed(x) if isinstance(x, list) else [x] for x in new_df], index=new_df.index)
new_df = new_df.dropna(how='all')
for col in new_df:
    df[col + 1] = df[col + 1].where(~switcher, new_df[col])

f = df[1].unique()
def temp(row):
    if row[3] in f and row[4] is not None:
        return f"{row[4]}.{row[3]}.{row[2]}.{row[1]}"
    elif row[2] in f and row[3] is not None:
        return f"{row[3]}.{row[2]}.{row[1]}"
    elif row[2] is not None:
        return f"{row[2]}.{row[1]}"
    else:
        return None
        # return '.'.join(map(str, row))
# sites0 = df.apply(temp, axis=1)
# sites = sites0.value_counts()

# d = load(file_name)
# print(f"{len(d.keys())} / {len(sites0.unique())}")
# print(f"{sites.loc[d.keys()].sort_values().sum()} / {sites0.shape[0]}")
# sites.loc[d.keys()].sort_values().head(10)

15509 / 131715
82883 / 199683


easykam.ru            1
mercury18.com         1
mercedesman.ru        1
mera-project.ru       1
stranao3.ru           1
goryachiekluchi.ru    1
megastar.su           1
stranaknig.com        1
megasport37.ru        1
medicamoscow.ru       1
dtype: int64

In [31]:
sites0 = df.apply(temp, axis=1)
sites = sites0.value_counts()

d = load(file_name)
print(f"{len(d.keys())} / {len(sites0.unique())}")
print(f"{sites.loc[d.keys()].sort_values().sum()} / {sites0.shape[0]}")
sites.loc[d.keys()].sort_values().head(10)

16559 / 131715
83933 / 199683


broda.ru            1
stroika19.ru        1
match-museum.ru     1
mct-oil.ru          1
mastikaizol.ru      1
mastertim.ru        1
masterpol-nsk.ru    1
mastermga.ru        1
masterkat.ru        1
matematikaege.ru    1
dtype: int64

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras

2023-02-18 16:45:00.277051: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-18 16:45:01.355805: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/suser/miniconda3/envs/tf/lib/:/home/suser/miniconda3/envs/tf/lib/python3.9/site-packages/tensorrt/:/home/suser/miniconda3/lib/python3.9/site_packages/tensorrt:/home/suser/miniconda3/lib
2023-02-18 16:45:01.355872: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open sh

In [2]:
from settings import path
from load import load, write

from a3 import get_common_feature, get_common_feature_dict, get_model, MyDataGenerator

In [3]:
pd.set_option('mode.chained_assignment', None)

In [4]:
TRY_NEW_MODEL = True
MODEL_SAVE_PATH = path['model']['approach_3']
FIT_DATA_FILE = MODEL_SAVE_PATH / 'fit_data_for_model.pickle'
FIT_DG_FILE = MODEL_SAVE_PATH / 'data_generator.pickle'

In [5]:
%%time
train_share = 0.995
dg = MyDataGenerator(train_share=train_share, target_type='train',
                     pca_var=0.99,
                     pca_sample_size=20000,
                     force_pca_fitting=False)
dg_valid = MyDataGenerator(train_share=train_share, target_type='test')
dg.data_shape, len(dg), len(dg_valid)

CPU times: user 2.51 s, sys: 7.9 s, total: 10.4 s
Wall time: 59.4 s


((93,), 263004, 1322)

In [6]:
if MODEL_SAVE_PATH.exists() and len(list(MODEL_SAVE_PATH.glob('*'))) > 2 and not TRY_NEW_MODEL:
    model = keras.models.load_model(MODEL_SAVE_PATH)
    fit_res = load(FIT_DATA_FILE, dict())
    print('model is loaded')
else:
    fit_res = dict()
    model = get_model(dg.data_shape,
                      ff_count=3, ff_activation='relu', ff_wide=None,
                      ff_h_count=1, ff_h_activation='elu', ff_h_wide=None,
                      dropout_h=None, dropout_v=None)

2023-02-18 16:46:02.242207: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:29:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-18 16:46:02.245265: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:29:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-18 16:46:02.245517: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:29:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-02-18 16:46:02.246007: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, 

In [7]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 93)]         0           []                               
                                                                                                  
 dense (Dense)                  (None, 93)           8742        ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 93)           8742        ['dense[0][0]']                  
                                                                                                  
 dense_2 (Dense)                (None, 93)           8742        ['dense_1[0][0]']                
                                                                                              

In [8]:
def plot(fit_res=fit_res):
    _, ax = plt.subplots(figsize=(20, 3))
    for name in ('binary_accuracy', 'auc'):
        ax.plot(fit_res[name])
    
    ax.plot(pd.Series(fit_res['binary_accuracy']).rolling(5).mean())
    ax.set_title(f"model accuracy {np.mean(fit_res['binary_accuracy'][-10:]):.3f}")
    ax.set_xlabel('epoch')
    plt.show()

In [9]:
%%time
with tf.device('/cpu:0'):
    model.fit(dg,
            batch_size=64,
            steps_per_epoch=1000,
            epochs=1)

2023-02-18 16:46:04.441641: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:417 : INVALID_ARGUMENT: Trying to access resource Resource-16-at-0x1ea58200 (defined @ /home/suser/miniconda3/envs/tf/lib/python3.9/site-packages/keras/engine/base_layer_utils.py:134) located in device /job:localhost/replica:0/task:0/device:GPU:0 from device /job:localhost/replica:0/task:0/device:CPU:0
 Cf. https://www.tensorflow.org/xla/known_issues#tfvariable_on_a_different_device
2023-02-18 16:46:04.441677: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at xla_ops.cc:417 : INVALID_ARGUMENT: Trying to access resource Resource-13-at-0x1ea39c00 (defined @ /home/suser/miniconda3/envs/tf/lib/python3.9/site-packages/keras/engine/base_layer_utils.py:134) located in device /job:localhost/replica:0/task:0/device:GPU:0 from device /job:localhost/replica:0/task:0/device:CPU:0
 Cf. https://www.tensorflow.org/xla/known_issues#tfvariable_on_a_different_device
2023-02-18 1

InvalidArgumentError: Graph execution error:

Detected at node 'StatefulPartitionedCall_16' defined at (most recent call last):
    File "/home/suser/miniconda3/envs/tf/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/suser/miniconda3/envs/tf/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/suser/miniconda3/envs/tf/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/suser/miniconda3/envs/tf/lib/python3.9/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/suser/miniconda3/envs/tf/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/home/suser/miniconda3/envs/tf/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/ho

In [10]:
%%time
with tf.device('/gpu:0'):
    model.fit(dg,
            batch_size=64,
            steps_per_epoch=1000,
            epochs=1)

2023-02-18 16:46:05.405860: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7ff1dc014c30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-02-18 16:46:05.405897: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 2060, Compute Capability 7.5
2023-02-18 16:46:05.409296: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-02-18 16:46:05.493065: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-02-18 16:46:05.519108: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1000/1000 [==============================] - 18s 16ms/step - loss: 0.8988 - binary_accuracy: 0.4960 - auc: 0.4957
CPU times: user 1min 19s, sys: 2min 56s, total: 4min 15s
Wall time: 17.7 s


In [11]:
# %%time
# with tf.device('/cpu:0'):
#     for _ in range(10):
#         history = model.fit(dg,
#                             batch_size=64,
#                             steps_per_epoch=1000,
#                             epochs=10,
#                             validation_data=dg_valid,
#                             validation_freq=20,
#                             verbose=1)
#         for k, v in history.history.items():
#             if k not in fit_res:
#                 fit_res[k] = []
#             fit_res[k] += v
#         plot()
# #     model.save(MODEL_SAVE_PATH)
# #     write(FIT_DATA_FILE, fit_res)
# # write(FIT_DG_FILE, dg)

In [12]:
# help(model.fit)